[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eneskelestemur/MolecularModeling/blob/main/Labs/lab08_molecular_dynamics/Molecular_dynamics.ipynb)

In this exercise, we'll use `openmm` and `mdanalysis` packages. Their recommended way of installation is through Ancaonda/Miniconda, so before starting this notebook prepare the environment using Anaconda. 

# Molecular Dynamics Using Openmm

In this lab, we will demonstrate Molecular Dynamics Simulation and its analysis in Python.

* [Estrogen-Raloxifene Example](#estrogen-raloxifene-example)
* [Ras-Raf Example](#ras-raf-example)